## Explore

This notebook is running in a `docker container` and being accessed from the docker host, `volumes are mapped` which means any changes here will change the file in the rpository

In [1]:
import os
import pandas as pd
os.chdir("..")

In [2]:
from src import create_app
from src.database import db
from src.models.phrase import Phrase
from src.models.language import Language
from src.models.rating import Rating
from src.models.phraseselection import PhraseSelection
from src.utils import query
from sqlalchemy import text

In [3]:
app = create_app(test=True)

In [30]:
with app.app_context():
    db.drop_all()

In [5]:
# write data to db
mock_phraseselection_data_1 = {"userid": "1", "phraseid": 1}
mock_phraseselection_data_4 = {"userid": "2", "phraseid": 4}

mock_phrase_data_1 = {"languageid": 2, "userid": "1",
                              "l1": "l1 phrase 1", "l2": "l2 phrase", "category": "restaurant"}
mock_phrase_data_2 = {"languageid": 2, "userid": "1",
                              "l1": "l1 phrase 2", "l2": "l2 phrase", "category": "restaurant"}
mock_phrase_data_3 = {"languageid": 2, "userid": "1",
                              "l1": "l1 phrase 3" , "l2": "l2 phrase", "category": "restaurant"}
mock_phrase_data_4 = {"languageid": 2, "userid": "1",
                              "l1": "l1 phrase 4", "l2": "l2 phrase", "category": "cafe"}
mock_phrase_data_5 = {"languageid": 2, "userid": "2",
                              "l1": "l1 phrase 4", "l2": "l2 phrase", "category": "restaurant"}

mock_rating_data_1 = {"phraseid": 1, "userid": "1", "rating": 1}
mock_rating_data_2 = {"phraseid": 1, "userid": "2", "rating": 2}
mock_rating_data_3 = {"phraseid": 1, "userid": "3", "rating": 4}
mock_rating_data_4 = {"phraseid": 1, "userid": "4", "rating": 5}
mock_rating_data_5 = {"phraseid": 2, "userid": "5", "rating": 1}
mock_rating_data_6 = {"phraseid": 2, "userid": "4", "rating": 3}
mock_rating_data_7 = {"phraseid": 3, "userid": "4", "rating": 5}
mock_rating_data_8 = {"phraseid": 3, "userid": "4", "rating": 3}


with app.app_context():
    for mock_phrase_selection_data in [mock_phraseselection_data_1, mock_phraseselection_data_4]:
            db.session.add(PhraseSelection(**mock_phrase_selection_data))
            db.session.commit()

    for mock_phrase_data in [mock_phrase_data_1, mock_phrase_data_2, mock_phrase_data_3, mock_phrase_data_4, mock_phrase_data_5]:
        db.session.add(Phrase(**mock_phrase_data))
        db.session.commit()
        
    for mock_rating_data in [mock_rating_data_1, mock_rating_data_2, mock_rating_data_3, 
                             mock_rating_data_4, mock_rating_data_5, mock_rating_data_6, 
                             mock_rating_data_7, mock_rating_data_8]:
        db.session.add(Rating(**mock_rating_data))
        db.session.commit()




In [11]:
with app.app_context():
    
    category="restaurant"
    userid="1"
    # (a) get phraseids in phraseselections for given userid
    # (b) filter for given category
    # (c) filter for phraseids not present from (a)
    # (d) join to phrases on phraseid
    # (e) join to ratings on phraseid
    # (f) groupby phraseid AVG(rating)
    
    data = query_pd(f"""
        SELECT phrases.phraseid, phrases.l1, phrases.l2, ROUND(AVG(ratings.rating), 0) AS average_rating
        FROM phrases 
        LEFT JOIN ratings
        ON ratings.phraseid = phrases.phraseid
        WHERE phrases.phraseid 
        NOT IN (SELECT DISTINCT(phraseid)
                FROM phraseselections
                WHERE userid = '{userid}')
        AND phrases.category = '{category}'
        GROUP BY phrases.phraseid
        ORDER BY average_rating DESC
        """, db)
    
data.head()

,phraseid,l1,l2,average_rating
0,2,l1 phrase 2,l2 phrase,None
1,3,l1 phrase 3,l2 phrase,None
2,5,l1 phrase 4,l2 phrase,None
